# [Metaprogramming](https://docs.julialang.org/en/v1/manual/metaprogramming/index.html)

## Expression

Julia represents its own code as a data structure of the language itself.
1. Every Julia program starts life as a string, and then the next step is to parse each string into an object called an expression: `Expr`.
1. `Expr` object has two parts:
    * a `Symbol`: identifys the kind of expression
    * the expression arguments: may be symbols, other expressions, or literal values

    ```julia
    # here type(ex) returns Expr
    ex.head    # returns Symbol part of ex1
    ex.args    # returns the expression argument part of ex1
    ```

1. two ways to construct `Expr`
    * by parsing
    * by direct construction from prefix notation

_**The key point here is that Julia code is internally represented as a data structure that is accessible from the language itself.**_

Expressions are just `Expr` objects which can be constructed programmatically and then evaluated, it is possible to dynamically generate arbitrary code which can then be run using `eval`.

In [ ]:
# Expr by parsing

prog = "1 + 1"
ex1 = Meta.parse(prog)

# Expr from prefix notation
ex2 = Expr(:call, :+, 1, 1)

@show ex1 == ex2

@show typeof(ex1)
@show ex1.head
@show ex1.args;

`:` in Julia has two functions:

1. _**create a `Symbol`**_
1. _**quoting: create expression objects without using the explicit `Expr` constructor**_
    * The usual representation of a quote form in an AST is an `Expr` with head `:quote:`

In [26]:
:foo  # this creates a JPL Symbol

typeof(ans)

Expr

In [23]:
ex = :(a+b*c+1)  # this creates an JPL expression

typeof(ex)

Expr

In [24]:
# Quoting for multiple expressions
ex = quote
        x = 1
        y = 2
        x + y
     end

quote
    #= In[24]:4 =#
    x = 1
    #= In[24]:5 =#
    y = 2
    #= In[24]:6 =#
    x + y
end

## Interpolation

Interpolation of literals or expressions into quoted expressions. Interpolation is indicated by a prefix `$`.

* Julia allows interpolation of literals or expressions into quoted expressions
* `$` interpolation syntax allows inserting only a single expression into an enclosing expression. 

## `Eval`

* Given an expression object, one can cause Julia to evaluate (execute) it at _**global scope**_ using `eval`.
    * Executing `eval` can also have side-effects that alter the state of the enclosing module's environment.
* Every [module](https://docs.julialang.org/en/v1/manual/modules/#modules-1) has its own `eval` function that evaluates expressions in its global scope.

## Functions on `Expr`

* A function can also take one or more Expr objects as arguments, and return another `Expr`.
* In JPL, it is possible to _**generate and manipulate Julia code within Julia itself**_.

## Macro

* A macro maps a tuple of arguments to a returned expression.
* The resulting expression is compiled directly rather than requiring a runtime `eval` call.
* Macros execute when code is parsed. They allow the programmer to generate and include fragments of customized code before the full program is run.

### Hygiene

When writing a macro, there are usually two concers:

1. Macros _**MUST**_ ensure that the variables they introduce in their returned expressions _**do not accidentally clash with existing variables**_ in the surrounding code they expand into.
1. A macro may be called in a different module from where it was defined. In this case we need to ensure that _**all global variables are resolved to the correct module**_.

How Julia solves the above problems?

1. Variables within a macro are classified as either local or global.
    * A variable is considered local if it is assigned to (and not declared global), declared local, or used as a function argument name.
    * Otherwise, it is considered global.
1. Local variables are then renamed to be unique
    * using the [gensym](https://docs.julialang.org/en/v1/base/base/#Base.gensym) function, which generates new symbols
1. Global variables are resolved within the macro definition environment.

Let's see an example:

```julia
macro time(ex)
    return quote
        local t0 = time()
        local val = $ex
        local t1 = time()
        println("elapsed time: ", t1-t0, " seconds")
        val
    end
end
```

Therefore the macro's locals will not conflict with any user variables, and `time` and `println` refer to the Julia Base definitions.

### Escape the macro hygiene

* The escaping mechanism can be used to "violate" hygiene when necessary, _**in order to introduce or manipulate user variables**_.
* Expression escaping is done with [esc](https://docs.julialang.org/en/v1/base/base/#Base.esc).

The following descriptions are from Julia's manual. I cannot understand it very well right now. Just paste them here.

>Getting the hygiene rules correct can be a formidable challenge.
>1. Before using a macro, you might want to consider whether a function closure would be sufficient.
>1. Another useful strategy is to defer as much work as possible to runtime.
>
>For example, many macros simply wrap their arguments in a QuoteNode or other similar Expr. Some examples of this include `@task body` which simply `returns schedule(Task(() -> $body))`, and `@eval expr`, which simply `returns eval(QuoteNode(expr))`.
